# TODO
1) Télécharger l'excel, lire l'onglet voulu --> paramètre
2) Extraire les données de l'onglet voulu et les mettre dans un grand tableau
3) Ajouter les calculs manquant pour les sorties manquantes (nombre de véhicule, coûts d'OPEX, CAPEX ?) --> lister les paramètres pour calculer ces sorties
4) Couper ce tableau et en répartir les données dans un excel, un onglet par ligne

A. tentative à partir du GTFS

In [1]:
import sys
sys.path.insert(0, r'../2e semestre 2023/quetzal')

from quetzal.io.gtfs_reader import importer
from quetzal.io import excel
from quetzal.model import stepmodel, model
from syspy.io.geojson_utils import gdf_to_geojson

from shapely.geometry import Point, MultiPoint, LineString
import geopandas as gpd
import pandas as pd
import numpy as np
import math

from shapely.ops import cascaded_union

In [2]:
file_name = 'gtfs-Al_Ula-Phase_3_-_Ambition_2035_-_South_Village_PR' #'gtfs-Al_Ula-Phase_3_-_2024_RATP_Dev' #gtfs-Al_Ula-Base_0.1_indicators_no_adjusted_capa_20231211
bus_buffer = 300
tram_buffer = 400
crs_parameter = 2318 #9360 #2318

In [3]:
gtfs = importer.GtfsImporter(path=r'./'+ file_name + '.zip', dist_units='m')
gtfs = gtfs.restrict_to_timerange(['05:00:00', '23:59:59'])
gtfs.describe()

,indicator,value
0,agencies,[Agency created by default]
1,running_services,"[Weekend, Weekday]"
2,timezone,Europe/Paris
3,start_date,20220314
4,end_date,20220318
5,num_routes,21
6,num_trips,84
7,num_stops,207
8,num_shapes,42
9,num_frequencies,572


In [4]:
emp = gpd.read_file(r'../GIS Data/emp.geojson')
pop = gpd.read_file(r'../GIS Data/pop.geojson')
tour = gpd.read_file(r'../GIS Data/tour.geojson')

# Couverture de tout le réseau :

In [5]:
# Associer un mode au stop
stops = gpd.GeoDataFrame(gtfs.stops, geometry=gpd.points_from_xy(gtfs.stops.stop_lon, gtfs.stops.stop_lat))

In [6]:
stops_coverage_table = pd.merge(
    pd.merge(
        pd.merge(gtfs.routes[['route_short_name', 'route_type', 'route_id']], gtfs.trips[['route_id', 'trip_id']], on = 'route_id'),
        gtfs.stop_times[['trip_id', 'stop_id']], on= 'trip_id'
    ), gtfs.stops, on= 'stop_id'
    )
stops_coverage_table = gpd.GeoDataFrame(stops_coverage_table)
stops_coverage_table = stops_coverage_table.set_crs(epsg = 4326)
stops_coverage_table = stops_coverage_table.to_crs(epsg=crs_parameter)
stops_coverage_table['geometry'] = stops_coverage_table.apply(
    lambda x: x.geometry.buffer(
        (x.route_type==0)*tram_buffer +
        (x.route_type!=0)*bus_buffer
    ), axis=1
)
# stops_coverage_table = stops_coverage_table[stops_coverage_table['route_short_name'] == 'Tram Phase 1']
stops_coverage_table = stops_coverage_table.to_crs(epsg = 4326)

In [7]:
# stops_coverage_table.head()

In [8]:
# stops_coverage_table = stops_coverage_table[stops_coverage_table['route_short_name'] == 'X1']
overall_coverage_geom = gpd.GeoSeries(cascaded_union(stops_coverage_table.geometry))
temp = pd.DataFrame()
overall_coverage = gpd.GeoDataFrame(temp, crs="EPSG:4326")
overall_coverage = overall_coverage.set_geometry(overall_coverage_geom)
overall_coverage

,geometry
0,"MULTIPOLYGON (((37.85022 26.36221, 37.84998 26..."


In [9]:
# stops_coverage_table['route_type'] = stops_coverage_table['route_type'].apply(lambda x: int(x))
# small_stops_coverage_table = stops_coverage_table[['stop_id', 'route_type', 'geometry']].sort_values(by = 'route_type', ascending = True)
# small_stops_coverage_table = small_stops_coverage_table.drop_duplicates(subset=['stop_id'], keep='first')
# # small_stops_coverage_table

In [10]:
pop_couv = 0
emp_couv = 0
tour_couv = 0

for i in range(0, len(pop)):
    row = gpd.GeoDataFrame(pop.loc[i]).T.set_crs(4326)
    merged = gpd.overlay(row , overall_coverage, how='intersection')
    pop_couv += ((merged.density*merged.to_crs(crs_parameter).geometry.area)/1000000).sum()

for i in range(0, len(emp)):
    row = gpd.GeoDataFrame(emp.loc[i]).T.set_crs(4326)
    merged = gpd.overlay(row , overall_coverage, how='intersection')
    emp_couv += ((merged.density*merged.to_crs(crs_parameter).geometry.area)/1000000).sum()

for i in range(0, len(tour)):
    row = gpd.GeoDataFrame(tour.loc[i]).T.set_crs(4326)
    merged = gpd.overlay(row , overall_coverage, how='intersection')
    tour_couv += ((merged.density*merged.to_crs(crs_parameter).geometry.area)/1000000).sum()

print(int(pop_couv), ' inhab.    ',
      int(emp_couv), ' jobs    ',
      int(tour_couv), ' tour'
)
# .apply(lambda x: int(round(x,0))).sum()

# TOUJOURS PAS OK, on compte 2 fois certains arrêts qui se superposent.
# il faut faire l'intersaction à chaque zone, puis sommer le tout.

# tram phase 1: 2747  inhab.     3325  jobs     6504  tour
# B3: 15399  inhab.     11212  jobs     2441  tour
# -0.75%, -0.71% -0.49%

83183  inhab.     56283  jobs     30150  tour


general information 
<br>
1) OK route_short_name<br>
2) OK type (dictionnaire avec mode)<br>
2 bis) OK nombre de sens<br>
3) OK longueur moyenne de la ligne (deux sens attention)<br>
4) NOK ---- Arrêt principaux (où un maximum du max pop+emp+tour est couvert, les 5 premiers ?)<br>
4 bis) NOK ----- Arrêt avec un maximum de connexions<br>
5) Distance moyenne entre les stops<br>
5 bis) Weekday working hours, Weekend working hours, Other working hours<br>
5 ter) working period<br>

In [11]:
capa_dict = dict(
    {
        0:['Tramway', 304],
        204:['Coach', 60],
        3:['Bus', 90],
        5:['Cable', 20],
        702:['Coach', 60],
        711:['Shuttle', 10],
        715:['DRT', 20]
    }
)
specific_capa_dict = dict(
    {
        'JTT tram': ['Tramway',142],
    }
)

In [12]:
# Stops and connexions with lines
stop_times_df = gtfs.stop_times
trips_df = gtfs.trips
routes_df = gtfs.routes
stops_df = gtfs.stops
stop_and_routes = pd.merge(stop_times_df, trips_df, on = 'trip_id').merge(routes_df, on = 'route_id').merge(stops_df, on = 'stop_id')
stop_and_routes = stop_and_routes[~stop_and_routes['route_short_name'].str.contains('DRT')]
stop_and_routes = stop_and_routes[~stop_and_routes['route_short_name'].str.contains('DRDO')]

stop_and_routes = stop_and_routes[['route_short_name', 'stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
stop_and_routes = stop_and_routes.drop_duplicates()
stop_and_transport = stop_and_routes.groupby('stop_id')['route_short_name'].apply(tuple).reset_index(name='Transport')
stop_and_routes = stop_and_routes.groupby(['stop_id', 'stop_name'])['stop_lat', 'stop_lon'].first().reset_index()
stop_and_transport = stop_and_transport.merge(stop_and_routes, on = ['stop_id'])
stop_and_transport = gpd.GeoDataFrame(stop_and_transport, geometry=gpd.points_from_xy(stop_and_transport.stop_lon, stop_and_transport.stop_lat))
# stop_and_transport = stop_and_transport[stop_and_transport['Transport'].str.find('GRT')<0]
stop_and_transport['Transport'] = stop_and_transport.Transport.astype('string')
stop_and_transport['Transport'] = stop_and_transport['Transport'].str.replace(",)", "", regex=False)
stop_and_transport['Transport'] = stop_and_transport['Transport'].str.replace(")", "", regex=False)
stop_and_transport['Transport'] = stop_and_transport['Transport'].str.replace("'", "", regex=False)
stop_and_transport['Transport'] = stop_and_transport['Transport'].str.replace("(", "", regex=False)
stop_and_transport
# stop_and_transport[stop_and_transport['Transport'].str.find(',')>0]
# stop_and_transport.to_file('stops_and_transport_' + file_name + '.geojson', drive = "GeoJSON")
stop_and_transport['Connexions'] = stop_and_transport.Transport.apply(lambda x : x.count(','))
# stop_and_transport = stop_and_transport[stop_and_transport.Connexions > 0]
stop_and_transport = stop_and_transport.sort_values(by='Connexions', ascending=False)
stop_and_transport = stop_and_transport.drop_duplicates(subset='geometry', keep='first')
stop_and_transport['PT lines'] = stop_and_transport['Connexions']+1
stop_and_transport.to_file('Connexions_map.shp')
# stop_and_transport

In [13]:
link_route_trips = pd.merge(gtfs.routes, gtfs.trips, on = 'route_id')

def find_mode(name):
    if name in specific_capa_dict:
        mode = specific_capa_dict[name][0]
    else:
        mode = capa_dict[link_route_trips[link_route_trips.route_short_name == name]['route_type'].iloc[0]][0]
    return (mode)

def find_capa(name):
    if name in specific_capa_dict:
        capa = specific_capa_dict[name][1]
    else:
        capa = capa_dict[link_route_trips[link_route_trips.route_short_name == name]['route_type'].iloc[0]][1]
    return (capa)

def find_direction_nr(name):
    return int(len(link_route_trips[link_route_trips.route_short_name == name])/2)

def convert_to_mn(hhmmss):
    return int(hhmmss[0:2])*60 + int(hhmmss[3:5]) + int(hhmmss[6:8])/60

def convert_to_hhmmss(mm):
    h = str(int(mm//60)).zfill(2)
    m = str(int(mm%60)).zfill(2)
    s = str(int((mm%60 - int(mm%60))*60)).zfill(2)
    return h +':' + m + ':' + s

freq = gtfs.frequencies
freq['start'] = freq.start_time.apply(convert_to_mn)
freq['end'] = freq.end_time.apply(convert_to_mn)

def find_period(name):
    period = link_route_trips[link_route_trips.route_short_name == name][['route_short_name','service_id']].drop_duplicates()
    period = period.groupby('route_short_name')['service_id'].apply(tuple).reset_index(name='service_id')
    period['service_id'] = period['service_id'].astype('string')
    period['service_id'] = period['service_id'].str.replace(",)", "", regex=False)
    period['service_id'] = period['service_id'].str.replace(")", "", regex=False)
    period['service_id'] = period['service_id'].str.replace("'", "", regex=False)
    period['service_id'] = period['service_id'].str.replace("(", "", regex=False)
    return period.service_id.iloc[0]

def find_working_hours(name, period):
    freq_df = freq.copy()
    trip_id_list = link_route_trips[link_route_trips.service_id == period].trip_id
    freq_df = freq_df[freq_df['trip_id'].isin(trip_id_list)]

    start = freq_df[(freq_df.trip_id.isin(link_route_trips[link_route_trips.route_short_name == name].trip_id))].sort_values(by = 'start', ascending = True).start_time.iloc[0]
    end = freq_df[(freq_df.trip_id.isin(link_route_trips[link_route_trips.route_short_name == name].trip_id))].sort_values(by = 'end', ascending = False).end_time.iloc[0]
    return start + ' - ' + end
    
def find_min_hw(name, period):
    freq_df = freq.copy()
    trip_id_list = link_route_trips[link_route_trips.service_id == period].trip_id
    freq_df = freq_df[freq_df['trip_id'].isin(trip_id_list)]
    
    freq_df = freq_df[(freq_df.trip_id.isin(link_route_trips[link_route_trips.route_short_name == name].trip_id))]
    min_hw = freq_df.headway_secs.min() #.sort_values(by = 'headway_secs', ascending = True).start_time.iloc[0]
    # end = freq_df.sort_values(by = 'end', ascending = False).end_time.iloc[0]
    return int(min_hw/60)

def find_stops_with_connexions(name):
    return stop_and_transport[stop_and_transport.Transport.str.contains(name)]


In [14]:
find_stops_with_connexions('JTT tram').iloc[0].Transport.split(',')

['X Airport', ' JTT tram', ' LRT', ' B3', ' B5']

0     Weekday
1     Weekend
2     Weekday
3     Weekend
4     Weekday
       ...   
79    Weekend
80    Weekday
81    Weekend
82    Weekend
83    Weekday
Name: service_id, Length: 84, dtype: object

In [28]:
name = 'B1'
link_route_trips_wd = link_route_trips[link_route_trips.service_id == "Weekday"]
st = gtfs.stop_times
st = st[st.trip_id.isin(link_route_trips_wd[link_route_trips_wd.route_short_name == name].trip_id)]
st = st[['trip_id', 'arrival_time', 'departure_time', 'departure_time', 'stop_sequence']].sort_values(by = ['trip_id', 'stop_sequence'])
st

,trip_id,arrival_time,departure_time,departure_time,stop_sequence
4,1b982d60-2fa9-11ee-9f9e-02420acd2a04_1,00:00:00,00:00:15,00:00:15,1
5,1b982d60-2fa9-11ee-9f9e-02420acd2a04_1,00:02:02,00:02:17,00:02:17,2
6,1b982d60-2fa9-11ee-9f9e-02420acd2a04_1,00:06:54,00:07:09,00:07:09,3
7,1b982d60-2fa9-11ee-9f9e-02420acd2a04_1,00:08:26,00:08:41,00:08:41,4
8,1b982d60-2fa9-11ee-9f9e-02420acd2a04_1,00:09:08,00:09:23,00:09:23,5
9,1b982d60-2fa9-11ee-9f9e-02420acd2a04_1,00:10:23,00:10:38,00:10:38,6
10,1b982d60-2fa9-11ee-9f9e-02420acd2a04_1,00:11:23,00:11:38,00:11:38,7
11,1b982d60-2fa9-11ee-9f9e-02420acd2a04_1,00:12:28,00:12:43,00:12:43,8
12,1b982d60-2fa9-11ee-9f9e-02420acd2a04_1,00:13:44,00:13:59,00:13:59,9
13,1b982d60-2fa9-11ee-9f9e-02420acd2a04_1,00:15:08,00:15:23,00:15:23,10


In [23]:
link_route_trips[link_route_trips.route_short_name == 'B1'].trip_id

0    1b982d60-2fa9-11ee-9f9e-02420acd2a04_1
1    1b982d60-2fa9-11ee-9f9e-02420acd2a04_2
2    f7f5c620-ccf0-11ee-932d-02420acb9207_1
3    f7f5c620-ccf0-11ee-932d-02420acb9207_2
Name: trip_id, dtype: object

In [15]:
lines_stats = pd.DataFrame()

lines_stats['Name'] = link_route_trips.route_short_name.unique()
lines_stats = lines_stats.sort_values(by = "Name", ascending = False)
lines_stats['Mode'] = lines_stats['Name'].apply(lambda x: find_mode(x))
lines_stats['Vehicle size/capacity'] = lines_stats['Name'].apply(lambda x: find_capa(x))
lines_stats['Directions'] = lines_stats['Name'].apply(lambda x: find_direction_nr(x))

gpd_shapes = gtfs.shapes
gpd_shapes = gpd.GeoDataFrame(gpd_shapes, geometry=gpd.points_from_xy(gpd_shapes.shape_pt_lon, gpd_shapes.shape_pt_lat))
gpd_shapes = gpd_shapes.merge(link_route_trips[['route_short_name', 'shape_id']], on = 'shape_id')
gpd_shapes = gpd_shapes.groupby(['shape_id', 'route_short_name'], as_index = False)['geometry'].apply(lambda x: LineString(x.tolist()))
gpd_shapes = gpd.GeoDataFrame(gpd_shapes, geometry='geometry')
gpd_shapes.rename(columns={"route_short_name":"Name"}, inplace=True)

final_tab = gpd_shapes.copy()
final_tab = final_tab.set_crs(crs=4326)
final_tab = final_tab.to_crs(epsg=crs_parameter)
final_tab['Line length (km)'] = final_tab['geometry'].length/1000
final_tab = final_tab.to_crs(crs=4326)
final_tab = final_tab[['Name', 'Line length (km)']]
final_tab = final_tab.groupby(['Name'], as_index = False).mean()
final_tab['Line length (km)'] = final_tab['Line length (km)'].apply(lambda x : (np.round(x,1)))
final_tab = final_tab[['Name', 'Line length (km)']]
lines_stats = lines_stats.merge(final_tab, on = 'Name')

lines_stats['Working period'] = lines_stats['Name'].apply(lambda x: find_period(x))

lines_stats['Main stops'] = ''

# look for working hours
lines_stats['Working hours wday'] = lines_stats['Name'].apply(lambda x: find_working_hours(x, 'Weekday'))
lines_stats['Working hours wend'] = lines_stats['Name'].apply(lambda x: find_working_hours(x, 'Weekend'))
# bug certains après 23h en fin
lines_stats['Minimum headway'] = lines_stats['Name'].apply(lambda x: find_min_hw(x, 'Weekday'))

link_route_trips_stops = link_route_trips.merge(gtfs.stop_times, on = 'trip_id')
link_route_trips_stops = link_route_trips_stops.merge(gpd_shapes, on = 'shape_id')
stop_tab = link_route_trips_stops.groupby(['Name'], as_index=False)['stop_sequence'].max()
stop_tab.rename(columns={'stop_sequence': 'Number of stops per direction'}, inplace = True)

link_route_trips_stops_end = link_route_trips_stops.merge(stop_tab, on = 'Name')

travel_t = link_route_trips_stops_end[link_route_trips_stops_end.stop_sequence == link_route_trips_stops_end['Number of stops per direction']]
travel_t['Number of stops per direction'] = travel_t['Number of stops per direction']
travel_t['arrival_time'] = travel_t['arrival_time'].apply(convert_to_mn)
travel_t = travel_t.groupby(['Name'])[['Number of stops per direction', 'arrival_time']].mean().reset_index()
travel_t['Travel time'] = travel_t['arrival_time'].apply(convert_to_hhmmss) 

lines_stats = lines_stats.merge(travel_t, on = 'Name')

lines_stats['Average distance between stops (m)'] = (lines_stats['Line length (km)']/(lines_stats['Number of stops per direction']-1)).apply(lambda x : int(np.round(x*1000/50,0)*50))
lines_stats['Average commercial speed (kmph)'] = (lines_stats['Line length (km)']/lines_stats['arrival_time']*60).apply(lambda x : int(np.round(x,0)))
lines_stats['+10%'] = lines_stats['arrival_time']*1.1
lines_stats['+3min'] = lines_stats['arrival_time']+3
lines_stats['travel_time_plus_layover'] = lines_stats[['+10%','+3min']].max(axis=1)
lines_stats['Number of vehicles'] = (lines_stats['travel_time_plus_layover']/lines_stats['Minimum headway']).apply(lambda x: int(np.ceil(x)*2))
lines_stats['Number of vehicles + reserve'] = (lines_stats['Number of vehicles']*1.1).apply(lambda x: int(np.ceil(x)))
lines_stats['Covered population (inhabitants)'] = 0
lines_stats['Covered jobs'] = 0
lines_stats['Covered touristic trips']= 0
lines_stats['Maximum PPHPD'] = 60/lines_stats['Minimum headway']*lines_stats['Vehicle size/capacity']

lines_stats = lines_stats[
    ['Name', 
    'Mode', 
    'Line length (km)', 
    'Number of stops per direction', 
    'Average distance between stops (m)', 
    # 'Main stops', 
    'Working period', 
    'Working hours wday',	
    'Working hours wend',
    'Travel time', 
    'Average commercial speed (kmph)', 
    'Vehicle size/capacity', 
    'Number of vehicles',
    'Number of vehicles + reserve', 
    # 'Average Annual mileage per vehicle',
    'Covered population (inhabitants)', 'Covered jobs', 'Covered touristic trips',
    # 'Connexions with other lines', 
    "Minimum headway",
    'Maximum PPHPD'

    ]
    ]
lines_stats
# HUB list with name of the line in it

,Name,Mode,Line length (km),Number of stops per direction,Average distance between stops (m),Working period,Working hours wday,Working hours wend,Travel time,Average commercial speed (kmph),Vehicle size/capacity,Number of vehicles,Number of vehicles + reserve,Covered population (inhabitants),Covered jobs,Covered touristic trips,Minimum headway,Maximum PPHPD
0,X1,Coach,33.3,11.0,3350,"Weekday, Weekend",05:00:00 - 23:40:00,05:00:00 - 23:40:00,01:06:47,30,60,26,29,0,0,0,6,600.0
1,X Airport,Coach,28.7,6.0,5750,"Weekday, Weekend",05:00:00 - 23:30:00,05:00:00 - 23:00:00,00:50:35,34,60,24,27,0,0,0,5,720.0
2,LRT,Tramway,36.8,28.0,1350,"Weekday, Weekend",05:00:00 - 23:30:00,05:00:00 - 23:00:00,01:05:47,34,304,26,29,0,0,0,6,3040.0
3,JTT tram,Tramway,22.1,17.0,1400,"Weekday, Weekend",05:00:00 - 23:30:00,05:00:00 - 23:00:00,00:39:59,33,142,16,18,0,0,0,6,1420.0
4,Harrat Cable transport,Cable,3.3,2.0,3300,"Weekday, Weekend",05:00:00 - 23:40:00,05:00:00 - 23:40:00,00:06:34,30,20,2,3,0,0,0,20,60.0
5,F2,Coach,12.6,22.0,600,"Weekday, Weekend",05:00:00 - 23:00:00,05:00:00 - 23:00:00,00:25:58,29,60,6,7,0,0,0,10,360.0
6,F1,Coach,9.7,12.0,900,"Weekday, Weekend",05:00:00 - 23:00:00,05:00:00 - 23:00:00,00:19:15,30,60,6,7,0,0,0,10,360.0
7,C6,Coach,29.8,2.0,29800,"Weekday, Weekend",05:00:00 - 23:00:00,05:00:00 - 23:00:00,00:34:45,51,60,8,9,0,0,0,10,360.0
8,C5,Coach,19.4,2.0,19400,"Weekday, Weekend",05:00:00 - 23:00:00,05:00:00 - 23:00:00,00:20:43,56,60,8,9,0,0,0,6,600.0
9,C4,Coach,22.6,16.0,1500,"Weekday, Weekend",05:00:00 - 23:00:00,05:00:00 - 23:00:00,00:39:41,34,60,10,11,0,0,0,10,360.0


In [16]:
lines_stats[lines_stats.Name == name].Mode.iloc[0]

NameError: name 'name' is not defined

In [ ]:
# écrire csv à chaque stations_df et en tirer les connexions et stations principales
# avoir la pop totale couverte par la ligne

link_route_trips_wd = link_route_trips[link_route_trips.service_id == "Weekday"]


for name in lines_stats.Name:
    stations_df = find_stops_with_connexions(name)
    if lines_stats[lines_stats.Name == name].Mode.iloc[0] == 'Tramway':
        buff = tram_buffer
    else:
        buff = bus_buffer
    stations_df = find_stops_with_connexions(name)
    stations_df[['stop_id','stop_name', 'Transport', 'stop_lat', 'stop_lon', 'geometry', 'Connexions', 'PT lines']].to_excel('connexions_' + name + '.xlsx')
    
    st = gtfs.stop_times
    st = st[st.trip_id.isin(link_route_trips_wd[link_route_trips_wd.route_short_name == name].trip_id)]
    st = st[['trip_id', 'arrival_time', 'departure_time', 'departure_time', 'stop_sequence']].sort_values(by = ['trip_id', 'stop_sequence'])
    st.to_excel('stop_times_' + name + '.xlsx')


    stops_coverage_table = stations_df.set_crs(epsg = 4326)
    stops_coverage_table = stops_coverage_table.to_crs(epsg=crs_parameter)
    stops_coverage_table['geometry'] = stops_coverage_table.apply((lambda x: x.geometry.buffer(buff)), axis=1)
    stops_coverage_table = stops_coverage_table.to_crs(epsg = 4326)

    coverage_geom = gpd.GeoSeries(cascaded_union(stops_coverage_table.geometry))
    temp = pd.DataFrame()
    coverage = gpd.GeoDataFrame(temp, crs="EPSG:4326")
    coverage = coverage.set_geometry(coverage_geom)

    p = 0
    e = 0
    t = 0

    for i in range(0, len(pop)):
        row = gpd.GeoDataFrame(pop.loc[i]).T.set_crs(4326)
        merged = gpd.overlay(row , coverage, how='intersection')
        p += ((merged.density*merged.to_crs(crs_parameter).geometry.area)/1000000).sum()

    for i in range(0, len(emp)):
        row = gpd.GeoDataFrame(emp.loc[i]).T.set_crs(4326)
        merged = gpd.overlay(row , coverage, how='intersection')
        e += ((merged.density*merged.to_crs(crs_parameter).geometry.area)/1000000).sum()

    for i in range(0, len(tour)):
        row = gpd.GeoDataFrame(tour.loc[i]).T.set_crs(4326)
        merged = gpd.overlay(row , coverage, how='intersection')
        t += ((merged.density*merged.to_crs(crs_parameter).geometry.area)/1000000).sum()


    lines_stats.at[lines_stats[lines_stats.Name == name].index[0],'Covered population (inhabitants)'] = int(p)
    lines_stats.at[lines_stats[lines_stats.Name == name].index[0],'Covered jobs'] = int(e)
    lines_stats.at[lines_stats[lines_stats.Name == name].index[0],'Covered touristic trips'] = int(t)


lines_stats.T.to_excel('lines_stats.xlsx')

In [ ]:
# Main stops et connexions à chaque hub

for name in lines_stats.Name:
    stations_df = find_stops_with_connexions(name)
    if lines_stats.Name == 'Tramway':
        buff = tram_buffer
    else:
        buff = bus_buffer
    for row in stations_df
    
    for 
    pop = 0
    emp = 0
    tour = 0


find_stops_with_connexions('B1').iloc[0].Transport.split(',')





name = 'B1'
stops_table = find_stops_with_connexions(name)



performance
<br>
5) travel time<br>
6) Average commercial speed<br>
7) Rolling stock capacity (dict)<br>
8) minimum number of vehicles<br>
9) total number of vehicles (+10% rounded up)<br>
10) average annual mileage (function with time table, first per day)<br>

coverage
<br>
11) pop
12) emp
13) tour
<br>

connexions
<br>
14) stops share with other lines, list stop_name // connexions
<br>

Capacity Analysis
<br>
15) minimum hw<br>
16) max pphpd<br>
17) -- not available -- model outputs - ridership<br>
17) -- not available -- model outputs - max pphpd<br>

In [ ]:
# Si la ligne n'a qu'une direction, la doubler !
trip_et_voie_rfn_pertinent = trip_et_voie_rfn_pertinent.append({"shape_id": trip_id, "possible_geometries": possible_geometry}, ignore_index=True)